In [ ]:
# Key indicators:
# - Percentage verzoeken langer dan 3 seconden ##
# - Gemiddelde tijd tot reactie ##
# Gebruik:
    # Grafiek verbruik door de dag heen ( opties tot custom range) ###
# GPU geheugen belasting: grafiek door de dag heen (opties tot customs range)
# Parallel verzoeken 
# aantal tokens per second in en out (current) (aggregated per minutes) (opties om custom aggregations and averages)

In [11]:
# !pip install ubiops
!pip install pandas

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.0 MB 7.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.0 MB 6.6 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/11.0 MB 6.4 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/11.0 MB 6.5 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.0 MB 6.6 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.0 MB 6.8 MB/s eta 0:00:01
   ------------------------------------ --- 10.0/11.0 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/12.8 MB 8.3 MB/s eta 0:00:02
   ---------- ----------------------------- 3.4/12.8 MB 8.5 MB/s eta 0:00:02
   ---------------- ----------------------- 5.2/12.8 MB 8.9 MB/s eta 0:00:01
   --------

In [13]:
import ubiops
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from dateutil import parser
API_TOKEN_chat = "Token 9155a7c8fef85ff102bf6c3dddf6b3deb51a7586"
API_TOKEN_poc = "Token 481b206efdcee52b165f011605263baea8d6319a"
# Custom time range selection (example: one day)
start_date = parser.parse("2025-07-22 09:00:00")
end_date = parser.parse("2025-07-24 09:00:00")
gemma_deployment_label_poc = "deployment_version_id:07736fa1-9999-44c1-9dbd-7de83f43663f"

In [14]:
def make_connection(API_TOKEN, HOST="https://api.demo.vlam.ai/v2.1"):
  configuration = ubiops.Configuration()
  configuration.api_key['Authorization'] = API_TOKEN
  configuration.host = HOST
  api_client = ubiops.ApiClient(configuration)
  api = ubiops.CoreApi(api_client)
  return api

In [15]:
DEPLOYMENT_METRICS = {
    "deployments.credits": {"unit": "credits (float)", "description": "Usage of Credits"},

    "deployments.instances": {"unit": "instances (float)", "description": "Average number of active deployment instances"},
    "deployments.instance_start_time": {"unit": "seconds (float)", "description": "Average duration from instance creation to start time"},

    "deployments.input_volume": {"unit": "bytes (int)", "description": "Volume of incoming data in bytes"},
    "deployments.output_volume": {"unit": "bytes (int)", "description": "Volume of outgoing data in bytes"},

    "deployments.memory_utilization": {"unit": "bytes (int)", "description": "Peak memory used during a request"},

    "deployments.requests": {"unit": "requests (int)", "description": "Number of requests made to the object"},
    "deployments.failed_requests": {"unit": "requests (int)", "description": "Number of failed requests made to the object"},
    "deployments.request_duration": {"unit": "seconds (float)", "description": "Average time in seconds for a request to complete"},

    "deployments.express_queue_size": {"unit": "items (int)", "description": "Average number of queued express requests"},
    "deployments.express_queue_time": {"unit": "items (int)", "description": "Average time in seconds for an express request to start processing"},

    "deployments.batch_queue_size": {"unit": "items (int)", "description": "Average number of queued batch requests"},
    "deployments.batch_queue_time": {"unit": "items (int)", "description": "Average time in seconds for a batch request to start processing"},

    "deployments.network_in": {"unit": "bytes (int)", "description": "Inbound network traffic for a deployment version"},
    "deployments.network_out": {"unit": "bytes (int)", "description": "Outbound network traffic for a deployment version"},

    # hier toepasselijk alleen bij 'poc' project
    "custom.completion_tokens": {"unit": "tokens (int)", "description": "Total number of completion tokens"},
    "custom.prompt_tokens": {"unit": "tokens (int)", "description": "Total number of prompt tokens"},
    "custom.total_tokens": {"unit": "tokens (int)", "description": "Total number of tokens"},
    "custom.token_count": {"unit": "tokens (int)", "description": "Total number of tokens"},

    "deployment_requests.cpu_utilization": {"unit": "% (float)", "description": "Average CPU utilization"},
    "deployment_requests.gpu_utilization": {"unit": "% (float)", "description": "Average GPU utilization"},
    "deployment_requests.memory_utilization": {"unit": "bytes (int)", "description": "Average memory utilization"},

    "pipelines.failed_requests": {"unit": "requests (int)", "description": "Number of failed pipeline requests to a pipeline version"},
    "pipelines.input_volume": {"unit": "bytes (int)", "description": "Input volume of a request"},
    "pipelines.object_failed_requests": {"unit": "requests (int)", "description": "Number of failed pipeline object requests"},
    "pipelines.object_requests": {"unit": "requests (int)", "description": "Number of pipeline object requests"}
}

def get_deployment_metric(project_name, metric_name, start_date, end_date, aggregation_period=60, unit=None, description=False):
    """
    Fetch a deployment metric from UbiOps and return as a DataFrame.

    Args:
        metric_name (str): Name of the metric (e.g., 'deployments.credits').
        start_date (datetime): Start of the time range.
        end_date (datetime): End of the time range.
        aggregation_period (int): Aggregation period in seconds (default: 60).
        unit (str, optional): Unit of the metric. If None, will be looked up automatically.
        description (bool, optional): If True, print the metric's description and unit.

    Returns:
        pd.DataFrame: DataFrame with columns ['timestamp', 'value', 'unit'].
    """
    metric_info = DEPLOYMENT_METRICS.get(metric_name, {})
    if unit is None:
        unit = metric_info.get("unit", "unknown")
    if description:
        print(f"Metric: {metric_name}")
        print(f"Unit: {unit}")
        print(f"Description: {metric_info.get('description', 'No description available.')}")
        print()
    api = make_connection(API_TOKEN_poc) if project_name == "poc" else make_connection(API_TOKEN_chat)
    response = api.time_series_data_list(
        project_name=project_name,
        metric=metric_name,
        start_date=start_date.strftime("%Y-%m-%d %H:%M:%S.%f"),
        end_date=end_date.strftime("%Y-%m-%d %H:%M:%S.%f"),
        aggregation_period=aggregation_period
    )
    return pd.DataFrame([
        {
            "timestamp": pd.to_datetime(dp["start_date"] if isinstance(dp["start_date"], str) else dp["start_date"].isoformat()),
            "value": dp["value"],
            "unit": unit
        }
        for dp in response.to_dict()['data_points']
    ])

In [16]:
api = make_connection(API_TOKEN_poc)
completion_tokens = api.time_series_data_list(
    project_name="poc",
    metric="custom.completion_tokens",
    start_date="2025-07-13 00:00:00.00",
    end_date="2025-07-23 09:00:19.65", # Max time range of 365 days
    aggregation_period = 3600, # 1 day
    labels=gemma_deployment_label_poc
)

df= pd.DataFrame([
        {
            "timestamp": pd.to_datetime(dp["start_date"] if isinstance(dp["start_date"], str) else dp["start_date"].isoformat()),
            "value": dp["value"]
        }
        for dp in completion_tokens.to_dict()['data_points']
    ])
df

,timestamp,value
0,2025-07-14 06:00:00+00:00,50.000000
1,2025-07-14 07:00:00+00:00,49.500000
2,2025-07-14 08:00:00+00:00,173.020833
3,2025-07-14 09:00:00+00:00,143.666667
4,2025-07-14 11:00:00+00:00,162.977679
...,...,...
60,2025-07-22 15:00:00+00:00,244.500000
61,2025-07-22 20:00:00+00:00,168.858974
62,2025-07-23 06:00:00+00:00,332.333333
63,2025-07-23 07:00:00+00:00,186.060000


In [17]:
from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['timestamp']
  ys = series['value']

  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('timestamp', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('timestamp')
_ = plt.ylabel('value')

ModuleNotFoundError: No module named 'seaborn'

In [18]:
api = make_connection(API_TOKEN_chat)
active_instances = api.project_instances_list(project_name = "chat")
active_instances.to_dict()['results']

[{'id': '50c39711-325a-487a-baa0-e42633d2b8a8',
  'status': 'running',
  'time_created': datetime.datetime(2025, 7, 22, 7, 20, 23, 325536, tzinfo=tzutc()),
  'time_updated': datetime.datetime(2025, 7, 22, 7, 21, 44, 927080, tzinfo=tzutc()),
  'instance_type': {'id': '32f9a8ba-b187-4200-953b-28870dc5b64c',
   'name': '65536mb_32vcpu_2xl40s',
   'display_name': '65536 MB + 32 vCPU + 2 x NVIDIA L40s'},
  'deployment': 'gemma-3-openai',
  'version': 'v1'},
 {'id': '386ec9a1-92e7-42c1-ac05-2f94a1d8f386',
  'status': 'running',
  'time_created': datetime.datetime(2025, 7, 22, 7, 20, 21, 457796, tzinfo=tzutc()),
  'time_updated': datetime.datetime(2025, 7, 22, 7, 21, 44, 83536, tzinfo=tzutc()),
  'instance_type': {'id': '32f9a8ba-b187-4200-953b-28870dc5b64c',
   'name': '65536mb_32vcpu_2xl40s',
   'display_name': '65536 MB + 32 vCPU + 2 x NVIDIA L40s'},
  'deployment': 'llama-3-3-4bit-awq-openai',
  'version': 'v1'},
 {'id': 'afb92fa4-837a-471c-9234-b4a07f44a4d6',
  'status': 'running',
  'ti

In [19]:
# Get the credit usage of all deployments in your project in intervals of a minute
get_deployment_metric("poc","deployments.credits", start_date, end_date, aggregation_period=3600, description=True)


Metric: deployments.credits
Unit: credits (float)
Description: Usage of Credits



,timestamp,value,unit
0,2025-07-22 09:00:00+00:00,0.007821,credits (float)
1,2025-07-22 10:00:00+00:00,0.007917,credits (float)
2,2025-07-22 11:00:00+00:00,0.007917,credits (float)
3,2025-07-22 12:00:00+00:00,0.007917,credits (float)
4,2025-07-22 13:00:00+00:00,0.007521,credits (float)
5,2025-07-22 14:00:00+00:00,0.007916,credits (float)
6,2025-07-22 15:00:00+00:00,0.007917,credits (float)
7,2025-07-22 16:00:00+00:00,0.007917,credits (float)
8,2025-07-22 17:00:00+00:00,0.007435,credits (float)
9,2025-07-22 18:00:00+00:00,0.007917,credits (float)


In [ ]:
!pip install ubiops dash pandas plotly seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 38.9 MB/s eta 0:00:00


# Nieuwe sectie